# Build custom model fallback strategy for Langchain Models



**🚀 Do this in 1 line of code with [reliableGPT](https://github.com/BerriAI/reliableGPT)**

`reliableGPT(openai.ChatCompletion.create)`

👉 [Replit](https://replit.com/@krrishdholakia/CustomModelFallback?v=1)

Contributions welcome 😊

# Environment Set-Up

In [ ]:
!pip install openai langchain

In [2]:
import openai, os
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

chat = ChatOpenAI(temperature=0)
chat.predict("Translate this sentence from English to French. I love programming.")

"J'adore la programmation."

# Implementing model fallback strategy

Here's how we'll do this:
- `Wrapper Function`: We'll wrap the OpenAI endpoint, to catch any errors that might occur
- `Fallback Strategy`: List of models we want to rotate through, in case one fails

In [3]:
import traceback
import copy
import random

def fallback_fn(fn, args, kwargs, fallback_strategy):
  try:
    for model in fallback_strategy:
      kwargs['model'] = model
      completion = fn(**kwargs)
      if completion != None:
        break
    return completion
  except:
    traceback.print_exc()
    return None

def throw_random_error():
  if random.random() <= 0.2:
      raise Exception("Random error occurred!")
  else:
      print("No error occurred.")

# Our Wrapper Function!!
def wrapper_fn(fn, fallback_strategy=["gpt-4", "gpt-3.5-turbo-16k", "gpt-3.5-turbo"]): # setting our default fallback strategy, in case OpenAI fails
  def wrapped_fn(*args, **kwargs):
    try:
      print("received request!")
      throw_random_error() # let's simulate an OpenAI error
      result = fn(*args, **kwargs)
    except Exception as e:
      traceback.print_exc()
      result = fallback_fn(fn, args, kwargs, fallback_strategy=fallback_strategy)
      if result == None:
        raise e
      else:
        print(f"🚀Recovered result: {result}")
    return result
  return wrapped_fn

## Wrapping our OpenAI call!

In [4]:
original_fn = openai.ChatCompletion.create # save the original reference (good practice)
openai.ChatCompletion.create = wrapper_fn(openai.ChatCompletion.create)

# 🚀 Test with 100 OpenAI Calls

In [ ]:
for _ in range(100):
  chat_completion = chat.predict("Hello world!")
  print(chat_completion)